In [1]:
from sklearn.externals import joblib
from my_data_preparation_kit import get_all_usertalk_data_from_corpus, normalize # ユーザ発話に対する処理
from my_data_preparation_kit import word_table_json_name, synonym_table_json_name, load_json_as_dict # JSONファイル周りの処理
from my_feature_vector_creator_kit import get_all_words_in_corpus
import time
import MeCab

svr_model = "tuned_rbf_svr_model.pkl"
constructed_corpus_path = "dialogue_corpus.db"
pn_ja_dict_path = "../pn_ja.dic"

# data_preparation.ipynbで保存したword_tableとsynonym_tableを読み込む
w_table = load_json_as_dict(word_table_json_name)       # print(convert_dict_to_json(word_table, 4))   -> {"u\...": [], ... }
s_table = load_json_as_dict(synonym_table_json_name) # print(word_table["食べる"])                  -> [235301]

In [2]:
# 学習(チューニング)済みのSVRのモデルを読み込む
t1 = time.time()
loaded_model = joblib.load(svr_model)
t2 = time.time()
print("Finished loading learned SVR model: about {} seconds.".format(t2 - t1))

Finished loading learned SVR model: about 0.2832016944885254 seconds.


In [3]:
# 単語極性辞書の読み込み
pn_table = {}
with open(pn_ja_dict_path) as file:
    for line in file:
        line = line.split(":")
        pn_table[line[0]] = float(line[3])
print("Finished loading pn_ja.dic.")

Finished loading pn_ja.dic.


In [4]:
# コーパス内の全てのユーザ発話のデータ：[ talk_data -> tuple, ... ]
# テストのためのユーザ発話を読み込む
all_usertalk_data = get_all_usertalk_data_from_corpus()
# コーパス内の全単語
all_words_in_corpus = get_all_words_in_corpus()
# コーパス内の全単語の重複無しバージョン（順序を保持している）
# 参考サイト：https://note.nkmk.me/python-list-unique-duplicate/
tt = time.time()
all_words_in_corpus_uniq = sorted(set(all_words_in_corpus), key = all_words_in_corpus.index)
tt2 = time.time()
print("Finished making all_words-list which removed duplication. (order of the list is kept): about {} seconds.".format(tt2 - tt))

Finished getting all user talks from dialogue_corpus.db: about 0.20914721488952637 seconds.
Finished getting all words from dialogue_corpus.db: about 0.8055751323699951 seconds.
Finished making all_words-list which removed duplication. (order of the list is kept): about 12.727441787719727 seconds.


In [5]:
# コーパス内の全単語とその同義語を、対応する列にまとめたリスト
# 対応する単語IDが無い or 同義語が無い場合は、その単語自体のみを含んだリストが格納される
t0 = time.time()
columns = []
for word in all_words_in_corpus_uniq:
    column = [word]
    wordids = []
    if word in w_table.keys(): # wordがユーザ発話に含まれる単語ならば
        wordids = w_table[word]
    if wordids != []: # wordに対応する単語IDがあれば
        for wordid in wordids:
            # wordidに対応する同義語をcolumnに追加
            synonyms = s_table[str(wordid)]
            column.extend(synonyms)
    columns.append(column)
t01 = time.time()
print("Finished making a list about words in corpus and their synonyms: about {} seconds.".format(t01 - t0))

Finished making a list about words in corpus and their synonyms: about 0.021014928817749023 seconds.


In [6]:
# my_feature_vector_creator_kit.pyのmake_svr_learning_featureと
# ほぼ同じ手順で数値表現に直す
inputted_talk = all_usertalk_data[8000][1]

m = MeCab.Tagger("-Owakati")
bow = [0] * len(columns) # 入力されたユーザ発話の特徴ベクトル
word_pn_scores = 0 # ユーザ発話の極性スコアの記録
words_in_inputted_talk = m.parse(inputted_talk).split() # ユーザ発話に含まれる単語のリスト
for word in words_in_inputted_talk:
    # word：ユーザ発話に含まれる単語
    # columns：コーパス内の全単語とその同義語を、対応する列にまとめたリスト
    for col_i, column in enumerate(columns):
        if word in column: # wordが、対応する列にまとめておいた単語と同義語のリストに含まれていたら
            bow[col_i] = 1

    if word in pn_table.keys(): # 1つのユーザ発話内の単語の極性スコアを加算
        word_pn_scores += pn_table[word]
else:
    # 1発話の単語について全て調べ終わったら、発話文の各単語の極性スコアの平均値を算出する
    # 小数点以下切り捨て防止のため分子にfloat()、0除算防止のため分母に+1
    word_pn_ave = float(word_pn_scores) / (len(words_in_inputted_talk) + 1)
    bow.append(word_pn_ave)

bow = [bow] # 二次元である学習データに合わせるため

In [7]:
# 数値表現に直したユーザ発話を用いて、そのユーザ発話に対する時受諾度合いを推定する
true_acceptance = all_usertalk_data[8000][3]

print("ユーザ発話 = {}, 予測値 = {}".format(inputted_talk, loaded_model.predict(bow)))

ユーザ発話 = お腹いっぱいで食べれないよ。, 予測値 = [ 2.04937768]


In [9]:
result = loaded_model.predict(bow)[0]
import numpy as np
print(result, int(np.round(result)))

2.04937767528 2
